In [3]:
import torch
import random
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import pad

In [4]:
def fix_seed():
    random.seed(0)
    np.random.seed(0)
    torch.cuda.manual_seed(0)
    torch.backends.cudnn.determenistic=True
    return True

fix_seed()

True

# Adding padding to the Input Image

In [39]:
def add_padding(input_images):
    from torch.nn.functional import pad
    """
    
    Description:
    вход сверточного слоя имеет следующую размерность: 
      - размер батча
      - число каналов
      - высота
      - ширина
      
    Example:
    2 input_images with size: 3*3*3 (C*H*W) --> finil size: (2*3*3*3) B*C*H*W 
    """
    return pad(input=input_images, pad=(1,1,1,1), mode='constant', value=0)

    

In [36]:
# from torch.nn.utils.rnn import pad_sequence
# from torch.nn.functional import pad

# x = torch.Tensor([
#     [1, 2,  3],
#     [4, 5,  6]
# ])
#torch.nn.functional.pad(input, pad, mode='constant', value=0)

#from torch.nn.functional import pad <-- allows to add paddings
a = torch.ones(2, 2)
c = torch.Tensor([[0, 0]])

pad(input=a, pad=(1,1,1,1), mode='constant', value=0)

tensor([[0., 0., 0., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 0., 0., 0.]])

# Calculating Output size of ConvLayer

Сверточный слой это массив фильтров.

Каждый фильтр имеет следующую размерность:
- число слоев во входном изображении (для RGB это 3)
- высота фильтра
- ширина фильтра

В ядре (кернеле) все фильтры имеют одинаковые размерность, поэтому ширину и высоту фильтров называют шириной и высотой ядра. Чаще всего ширина ядра равна высоте ядра, в таком случае их называют размером ядра (kernel_size).

Также слой имеет такие параметры:
 - padding - на какое количество пикселей увеличивать входное изображение с каждой стороны.
 - stride - на сколько пикселей смещается фильтр при вычислении свертки

In [27]:
def calculate_conv_layer_output_size(input_matrix_shape,
                                     out_channels,
                                     kernel_size,
                                     stride,
                                     padding):
    """
    Input:
    - входную размерность(число изображений в батче*число слоев в одном изображении*высота изображения*ширина изображения)
    количество фильтров
    - размер фильтров (считаем, что высота совпадает с шириной)
    - padding
    - stride
    
    Output:
    
    """
    N_in = input_matrix_shape[0]
    C_in = input_matrix_shape[1]
    H_in = input_matrix_shape[2] #10
    W_in = input_matrix_shape[3] #10
    
    
    N_out = N_in
    C_out = out_channels
    H_out = ((H_in + 2 * padding - 1 * (kernel_size - 1) - 1) // stride) + 1
    W_out = ((W_in + 2 * padding - 1 * (kernel_size - 1) - 1) // stride) + 1
    result = torch.Tensor([
        N_out,
        C_out,
        H_out,
        W_out
    ])
    return result

In [29]:
input_matrix_shape=[2, 3, 10, 10] #2 images, RGB, height 10, width 10
out_channels=10
kernel_size=3
stride=1
padding=0

calculate_conv_layer_output_size(input_matrix_shape,
                                 out_channels,
                                 kernel_size,
                                 stride,
                                 padding)


tensor([ 2., 10.,  8.,  8.])